In [8]:
import pandas as pd
from api import get_api
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
from tqdm import tqdm

In [9]:
# Cluster vrp 정보 가져오기
solution = pd.read_csv('solution_cluster_info.csv')
delete = []
for i in range(len(solution)):
    if(solution.iloc[i,2] == 50.0):
        delete.append(i)
print(solution)
delete.sort(reverse=True)
print(delete)
for i in delete:
    solution = solution.drop([solution.index[i]])
    
#     
solution = pd.DataFrame(solution)
solution = solution.reset_index(drop=True)
solution

    Unnamed: 0   0   1     2     3     4     5     6     7     8     9    10
0            0  50  50   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
1            1  50   1  41.0  23.0  39.0  50.0   NaN   NaN   NaN   NaN   NaN
2            2  50  15  20.0  22.0   6.0  43.0  35.0  50.0   NaN   NaN   NaN
3            3  50  18  40.0   4.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN
4            4  50  19  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
5            5  50  45  49.0  26.0  38.0   0.0  27.0  37.0   8.0  50.0   NaN
6            6  50  31  14.0   7.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN
7            7  50  46  17.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN
8            8  50  32  13.0   9.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN
9            9  50   5  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
10          10  50  50   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
11          11  50  16  25.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,1,50,1,41.0,23.0,39.0,50.0,NaN,NaN,NaN,NaN,NaN
1,2,50,15,20.0,22.0,6.0,43.0,35.0,50.0,NaN,NaN,NaN
2,3,50,18,40.0,4.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,50,19,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,50,45,49.0,26.0,38.0,0.0,27.0,37.0,8.0,50.0,NaN
5,6,50,31,14.0,7.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
6,7,50,46,17.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,50,32,13.0,9.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
8,9,50,5,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11,50,16,25.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# 지점 정보 가져오기
spot = pd.read_csv('clusterinfo.csv')

In [11]:
def find(cNum):
    cNum = cNum[2:]
    result = pd.DataFrame()
    for i in cNum:
        if(i==50):
            break
        temp = spot[spot['cluster']==i]
        result = pd.concat([result, temp],ignore_index=True,axis=0)
        result = result[['lat','lng']]
    return result

In [12]:
solutionList = solution.values.tolist()
arrive = {'lat': 35.229546, 'lng' : 126.847516}
print(len(solutionList))

13


In [13]:
def calcdistance(A,B):
    return abs(A['lat']-B['lat']) + abs(A['lng']-B['lng'])

In [14]:
answer = []

sL = tqdm(solutionList)
for i in sL:
    newDF = pd.DataFrame(find(i))
    if(len(newDF) <= 0):
        continue
    newDF = newDF.append(arrive,ignore_index=True)
    result = []
    for j in range(len(newDF)-1):
        temp = []
        for k in range(len(newDF)):
            distance = calcdistance(newDF.iloc[j],newDF.iloc[k]) * 10000
            temp.append(distance)
        result.append(temp)
    result.append([0]*len(newDF))
    print(len(result),len(result[0]),len(result[len(result)-1]))
    
    
    def create_data_model():
        data = {}
        data['distance_matrix'] = result
        data['num_vehicles'] = 3
        data['depot'] = len(result) -1 
        return data
    
    # Instantiate the data problem.

    data = create_data_model()
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        1000000000,  # no slack
        3000000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    def print_solution_only_num():
        output = []
        for vehicle_id in range(data['num_vehicles']):
            Output = []
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                Output.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            Output.append(manager.IndexToNode(index))  
            output.append(Output)
        return output
    
    if solution:
        sol = print_solution_only_num()
        answer.append(sol)
    else:
        print('No solution found !')

print(answer)

  0%|                                                    | 0/13 [00:00<?, ?it/s]/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/1084501470.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)


219 219 219


  8%|███▍                                        | 1/13 [00:02<00:25,  2.14s/it]/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/1084501470.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)
 15%|██████▊                                     | 2/13 [00:03<00:16,  1.52s/it]

159 159 159


/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/1084501470.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)
 23%|██████████▏                                 | 3/13 [00:04<00:15,  1.58s/it]

...

160 160 160


/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/1084501470.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)
 69%|██████████████████████████████▍             | 9/13 [00:09<00:03,  1.29it/s]

92 92 92


/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/1084501470.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)
 69%|██████████████████████████████▍             | 9/13 [00:10<00:04,  1.15s/it]


KeyboardInterrupt: 

In [ ]:
sL = tqdm(solutionList)
for i in sL:
    arrive = {'lat': 35.229546, 'lng' : 126.847516}

    newDF = pd.DataFrame(find(i))
    
    if(len(newDF) <= 0):
        continue
        
    newDF = newDF.append(arrive,ignore_index=True)

    print(newDF)


In [ ]:
answer = pd.DataFrame(answer)
answer

In [15]:
answerDF = pd.read_csv('allDF.csv')

In [16]:
answerDF

,0,1,2
0,"[218, 115, 116, 113, 114, 83, 91, 106, 20, 33,...","[218, 112, 60, 55, 218]","[218, 217, 171, 201, 198, 194, 188, 179, 178, ..."
1,"[158, 2, 1, 0, 156, 155, 154, 153, 39, 43, 41,...","[158, 5, 31, 23, 12, 25, 17, 16, 29, 10, 4, 7,...","[158, 157, 136, 158]"
2,"[195, 158, 147, 163, 167, 181, 152, 178, 189, ...","[195, 186, 175, 179, 173, 106, 107, 56, 36, 41...","[195, 194, 172, 190, 176, 156, 154, 160, 166, ..."
3,"[40, 12, 14, 16, 22, 6, 9, 18, 20, 38, 35, 32,...","[40, 1, 13, 3, 4, 7, 40]","[40, 26, 39, 36, 21, 15, 28, 27, 40]"
4,"[219, 203, 199, 174, 193, 202, 175, 166, 207, ...","[219, 13, 14, 216, 178, 184, 4, 3, 219]","[219, 0, 1, 2, 9, 11, 7, 5, 6, 8, 10, 12, 57, ..."
5,"[128, 63, 60, 54, 16, 17, 18, 24, 29, 32, 35, ...","[128, 95, 110, 90, 0, 3, 9, 11, 10, 5, 4, 1, 1...","[128, 23, 39, 40, 59, 74, 37, 51, 69, 28, 31, ..."
6,"[89, 7, 14, 1, 4, 5, 2, 9, 11, 89]","[89, 0, 3, 6, 8, 12, 13, 10, 89]","[89, 54, 71, 19, 86, 88, 62, 51, 80, 78, 64, 4..."
7,"[159, 156, 153, 151, 144, 139, 132, 126, 111, ...","[159, 101, 85, 89, 93, 96, 100, 94, 88, 90, 82...","[159, 99, 98, 95, 84, 83, 103, 102, 91, 92, 87..."
8,"[91, 37, 36, 34, 71, 45, 48, 58, 63, 89, 90, 1...","[91, 21, 88, 2, 5, 6, 9, 10, 11, 33, 1, 13, 26...","[91, 31, 66, 53, 43, 38, 25, 17, 16, 80, 70, 8..."
9,"[142, 83, 88, 74, 84, 86, 89, 95, 136, 97, 75,...","[142, 119, 130, 124, 122, 117, 116, 114, 111, ...","[142, 57, 11, 19, 37, 56, 67, 12, 28, 35, 40, ..."


In [17]:
m = folium.Map( location = [35.212616,126.842246],zoom_start=13)

solList = []
for i in solutionList:
    if(i[2] == 50.0):
        continue
    solList.append(i)
    
solutionList = solList

In [18]:
# sL = tqdm(solutionList)
color = ['blue','red','yellow']
for i in range(len(answerDF)):

    for j in range(len(answerDF.iloc[i])):

        tempList = answerDF.iloc[i,j]
        tempList = tempList[1:]
        tempList = tempList[:-1]
        tempList = tempList.replace(" ","")
        tempList = tempList.split(',')

        if(len(tempList) <= 2):
            continue        
            
        newDF = pd.DataFrame(find(solutionList[i]))
        if(int(len(newDF)) <= 0):
            continue


        newDF = newDF.append(arrive,ignore_index=True)
#         tempList.append(len(newDF)-1)
        for k in range(1,len(tempList)-1):
            if k == 1:
                folium.Marker([newDF.iloc[int(tempList[k])]['lat'],newDF.iloc[int(tempList[k])]['lng']]).add_to(m)
            folium.PolyLine(locations=[[newDF.iloc[int(tempList[k])]['lat'],newDF.iloc[int(tempList[k])]['lng']], [newDF.iloc[int(tempList[k])+1]['lat'],newDF.iloc[int(tempList[k])+1]['lng']]]
                            ,tooltip='Polyline'
                            ,color=color[j]
                           ).add_to(m)


/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/3705004957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/3705004957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/3705004957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF = newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2743/3705004957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [19]:
m